# Process Data from 2014 into a consistent format.

> This notebook brings the 2014 into alignment with the desired format with respect to field name, type, and grouping.

In [ ]:
# Imports ----
import re
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
import pickle

In [ ]:
from g2fd.internal import *

In [ ]:
#| default_exp internal

In [ ]:
# 2014
year_string = '2014'

meta_path = './data/raw/G2F_Planting_Season_2014_v4/z._2014_supplemental_info/g2f_2014_field_characteristics.csv' 
phno_path = './data/raw/G2F_Planting_Season_2014_v4/a._2014_hybrid_phenotypic_data/g2f_2014_hybrid_data_clean.csv' 
# geno_path = None,  
wthr_path = './data/raw/G2F_Planting_Season_2014_v4/b._2014_weather_data/g2f_2014_weather.csv'
          # no soil 2014, some info in metadata
# soil_path = None, 
          # no agro 2014, some info in metadata
# mgmt_path = None,



meta = pd.read_csv(meta_path, encoding = "ISO-8859-1", low_memory=False)
phno = pd.read_csv(phno_path, encoding = "ISO-8859-1", low_memory=False)
wthr = pd.read_csv(wthr_path, encoding = "ISO-8859-1", low_memory=False)
# soil = pd.read_csv(soil_path, encoding = "ISO-8859-1", low_memory=False)
# mgmt = pd.read_csv(mgmt_path, encoding = "ISO-8859-1", low_memory=False)

In [ ]:
# load dicts for column renaming
meta_name_dict = mk_name_dict(name = 'meta')
phno_name_dict = mk_name_dict(name = 'phno')
# soil_name_dict = mk_name_dict(name = 'soil')
wthr_name_dict = mk_name_dict(name = 'wthr')
# mgmt_name_dict = mk_name_dict(name = 'mgmt')

# Rename
**Naming rules:**
- One dict for each input df
- Comment out anything that shouldn't be changed
- Upper_Upper_Unit_\$unit
- Upper_$number
- No special characters


In [ ]:
(find_unrecognized_columns(df = meta, dct = meta_name_dict),
find_unrecognized_columns(df = phno, dct = phno_name_dict),
# find_unrecognized_columns(df = soil, dct = soil_name_dict),
find_unrecognized_columns(df = wthr, dct = wthr_name_dict)#,
# find_unrecognized_columns(df = mgmt, dct = mgmt_name_dict)
)

([], [], [])

In [ ]:
# prlst2dct([])

meta.loc[:, ['Location name',
  'Soil test type',
  'Soil texture',
  'Soil pH']].head(3)

,Location name,Soil test type,Soil texture,Soil pH
0,DE,NaN,NaN,NaN
1,GA,See file 'Additional G2F metadata-GA1-2014.xlsx',Loamy Sand,7.1
2,IA1,NaN,NaN,NaN


In [ ]:
# meta = meta.rename(columns=meta_name_dict)
# phno = phno.rename(columns=phno_name_dict)
# soil = soil.rename(columns=soil_name_dict)
# wthr = wthr.rename(columns=wthr_name_dict)
# mgmt = mgmt.rename(columns=mgmt_name_dict)

# # add indicator columns to help with debugging merge
# meta['meta'] = True
# phno['phno'] = True
# soil['soil'] = True
# wthr['wthr'] = True
# mgmt['mgmt'] = True

In [ ]:
[e.shape for e in [meta, phno, wthr]]

[(42, 42), (12675, 38), (207004, 24)]

# Sanatize ID columns as needed


In [ ]:
# soil = sanitize_Experiment_Codes(
#     df = soil, 
#     simple_renames = {
#         'W1H1': 'WIH1', 
#         'W1H2': 'WIH2', 
#         'W1H3': 'WIH3'
#     }, 
#     split_renames = {
#         'NEH2_NEH3': ['NEH2', 'NEH3']
#     })

# wthr = sanitize_Experiment_Codes(
#     df = wthr, 
#     simple_renames = {
#     }, 
#     split_renames = {
#         'NEH2_NEH3': ['NEH2', 'NEH3'],
#         'NYH3_NYS1': ['NYS1', 'NYH3'],
#         'TXH1_TXH3': ['TXH1', 'TXH3']
#     })

In [ ]:
# # confirm everything's okay
# print(
#   'meta', find_unrecognized_experiments(meta.Experiment_Code, return_all_exps=False), 
# '\nphno', find_unrecognized_experiments(phno.Experiment_Code, return_all_exps=False),
# '\nsoil', find_unrecognized_experiments(soil.Experiment_Code, return_all_exps=False),
# '\nwthr', find_unrecognized_experiments(wthr.Experiment_Code, return_all_exps=False),
# '\nmgmt', find_unrecognized_experiments(mgmt.Experiment_Code, return_all_exps=False),
# '\nall ', find_unrecognized_experiments([], return_all_exps=True)
# )  

In [ ]:
# # Find minimum cols needed to index all rows
# df = phno
# id_cols = ['Year', 'Experiment_Code', 'Range', 'Pass', 'Plot',]
# candidate_cols = ['State', 'City',
#                  'Experiment', 'Source', 'Pedigree', 'Family', 'Tester', 'Replicate',
#                   'Block',  'Plot_ID']
# target = df.shape[0]

# output = pd.DataFrame(zip(
#     candidate_cols,
#     [df.loc[:, id_cols+[e]].drop_duplicates().shape[0] for e in candidate_cols]
#    ), columns=['Additional_ID', 'Uniq_Vals'])

# output.assign(At_Target=lambda x:x.Uniq_Vals == target)


# Rearrange columns

In [ ]:
# # separate static and dynamic values
# sval = phno.merge(soil, how = 'outer')
# sval = sval.merge(meta, how = 'outer') # This introduces 3 sites that have no data
# # sval.shape # used to confirm nrow = #20574 + 3

# # these tables are different enought we'll keep them separate
# # mgmt
# # unfortunately we need multiples because at least one field treats different passes differently
# mgmt = phno.loc[:, ['Year', 'Experiment_Code', 'Range', 'Pass', 'Plot', 'phno']
#                ].drop_duplicates().merge(mgmt, how = 'outer')
# # confirm there are no rows in mgmt that are not in phno
# temp = mgmt.loc[(~mgmt.phno & mgmt.mgmt), :]
# if 0 != temp.shape[0]:
#     print(temp)
# else:
#     mgmt = mgmt.loc[mgmt.mgmt.notna(), :].drop(columns = 'phno')


# # wthr
# # There's only ever one weather station so we have to worry about imputation but not duplicates

In [ ]:
# # Set each id col to a string
# for i in ['Year', 'Experiment_Code', 'Range', 'Pass', 'Plot']:
#     sval[i] = sval[i].astype('string')
#     mgmt[i]  =  mgmt[i].astype('string')
    
#     if i not in ['Range', 'Pass', 'Plot']:
#         wthr[i]  =  wthr[i].astype('string')

# Sanitize Non-ID columns



## Sanitization functions

The pattern to use is:
 1. Alter the dataframe
 1. Test the dataframe against expectations
 
The main tasks that need to be completed are:
 1. Identify values that can't be converted to the expected data type. The "find_unconvertable_" family of functions should be used. 
     1. `find_unconvertable_datetimes`
     
 1. For simple renaming (e.g. misspellings) or splitting non-tidy data into two rows ("entry1-entry2" -> "entry1", "entry2") use `sanitize_col` 
 1. Move values that are ambigous but pertain to data imputation to "Imputation_Notes" using `relocate_to_Imputation_Notes`
 1. If new columns need to be added (e.g. mgmt.Ingredient for parsed components of Product (e.g. elements) ) this should be accomplished with `safe_create_col`.
 1. Any one off changes should be accomplised manually. 
 1. Confirm columns match the expected types with `check_df_dtype_expectations`, and report mismatches. 


These steps should be completed for each dataframe in turn to minimize the cognitive load of the reader. 

## Sanitization: Column data type expectations
Note: to handle missing values some columns that would otherwise be ints are floats

In [ ]:
# sval_col_dtypes = mk_dtype_dict(name = 'sval')
# wthr_col_dtypes = mk_dtype_dict(name = 'wthr')
# mgmt_col_dtypes = mk_dtype_dict(name = 'mgmt')

# Sanitization: Alter entries

## Static values (within season)

### Datetime containing columns

In [ ]:
# # convert the date cols into datetime. Lean on pd.to_datetime() to infer the format, assume that each site uses the same format.

# for e in ['Planted_Unit_Datetime', 
#     'Harvested_Unit_Datetime', 
#     'Anthesis_Unit_Datetime', 
#     'Silking_Unit_Datetime', 
#     'Recieved_Date_Unit_Datetime', 
#     'Processed_Date_Unit_Datetime', 
#     'Weather_Station_Placed_Unit_Datetime', 
#     'Weather_Station_Removed_Unit_Datetime'
#     ]:
# # find_unconvertable_datetimes(df_col=sval[e], pattern='%Y-%m-%d %H:%M', index=False)

#     sval['Datetime_Temp'] = pd.to_datetime(np.nan)

#     for code in list(sval.Experiment_Code.drop_duplicates()):
#     # code = list(sval.Experiment_Code.drop_duplicates())[0]
#         sval.loc[sval.Experiment_Code == code, 'Datetime_Temp'
#                  ] = pd.to_datetime(sval.loc[sval.Experiment_Code == code, e])

#     sval.loc[:, e] = sval.loc[:, 'Datetime_Temp'] 

# sval = sval.drop(columns = 'Datetime_Temp')

In [ ]:
# # # -> floats

# # # [find_unconvertable_numerics(df_col = sval[e], index = False) for e in [
# # #     'Alley_Length_Unit_Inches',
# # # 'Row_Spacing_Unit_Inches',
# # # 'Pounds_Needed_Soil_Moisture'
# # # ]]

# sval = sanitize_col(
#     df = sval, 
#     col = 'Pounds_Needed_Soil_Moisture', 
#     simple_renames= {'3 to 4':'3.5'}, 
#     split_renames= {})

In [ ]:
# # convert types
# for e in ['Alley_Length_Unit_Inches', 'Row_Spacing_Unit_Inches', 'Pounds_Needed_Soil_Moisture',
#          'Anthesis_Unit_Days', 'Silking_Unit_Days', 'Kernels_Per_Plot']:
#     err_list = find_unconvertable_numerics(df_col = sval[e], index = False)
#     if err_list != []:
#         print(e)
#         print(err_list)
#     else:
#         sval[e] = sval[e].astype('float')

In [ ]:
# # to bool
# sval = sanitize_col(
#     df = sval, 
#     col = 'Discarded', 
#     simple_renames= {
#         'Yes':'True',
#         'yes':'True'}, 
#     split_renames= {})

# # set missing to false
# sval.loc[sval.Discarded.isna(), 'Discarded'] = 'False'
# sval.Discarded = sval.Discarded.map({'True': True, 'False': False})

### Simple Columns

In [ ]:
# to float
# sval.Pounds_Needed_Soil_Moisture.astype(float)

In [ ]:
# # to bool
# sval['phno'] = sval['phno'].astype('bool')
# sval['soil'] = sval['soil'].astype('bool')
# sval['meta'] = sval['meta'].astype('bool')

# # to string
# sval = cols_astype_string(
#     df = sval, 
#     col_list = [key for key in sval_col_dtypes.keys() if sval_col_dtypes[key] == 'string'])

# sval.Year = year_string
# sval.Year = sval.Year.astype('string')

### Check Success

In [ ]:
# checkpoint = check_df_dtype_expectations(df = sval, dtype_dct = sval_col_dtypes)

# if sum(checkpoint.Pass)/checkpoint.shape[0] == 1:
#     pass
# else:
#     print(checkpoint.loc[~checkpoint.Pass, ]) 
#     print()

## Weather

### Datetime

In [ ]:
# # instead of writing regexes to figure out the mose likely format for each datetime, we assume each experiment will be consistent withing that experiment
# # and let pd figure it out.
# # wthr['Datetime_Temp'] = pd.to_datetime(np.nan)

# # for code in list(wthr.loc[:, 'Experiment_Code'].drop_duplicates()):
# #     wthr.loc[wthr.Experiment_Code == code, 'Datetime_Temp'] = pd.to_datetime(wthr.loc[wthr.Experiment_Code == code, 'Datetime'], errors='coerce')


# # ... or we use the fields in the df to make a consistent format
# wthr = cols_astype_string(
#     df = wthr, 
#     col_list = ['Year', 'Month', 'Day', 'Time'])

# wthr = sanitize_col(
#     df = wthr,
#     col = 'Time', 
#     simple_renames= {'24:00:00': '00:00:00'}, # this could be day + 24 h instead of a miscoded day + 0 h
#     split_renames= {})

# wthr['Datetime_Temp'] = wthr['Year']+'-'+wthr['Month']+'-'+wthr['Day']+' '+wthr['Time']

# # convert types
# err_list = find_unconvertable_datetimes(df_col=wthr['Datetime_Temp'], pattern='%Y-%m-%d %H:%M', index=False)
# if err_list != []:
#     print(err_list)
# else:
#     wthr.Datetime_Temp = pd.to_datetime(pd.Series(wthr.Datetime_Temp), errors='coerce')
#     wthr.Datetime = wthr.Datetime_Temp
#     wthr = wthr.drop(columns= 'Datetime_Temp')

### Simple Columns 

In [ ]:
# # to string
# wthr = cols_astype_string(
#     df = wthr, 
#     col_list = [key for key in wthr_col_dtypes.keys() if wthr_col_dtypes[key] == 'string'])

# wthr.Year = year_string
# wthr.Year = wthr.Year.astype('string')

### Check Success

In [ ]:
# checkpoint = check_df_dtype_expectations(df = wthr, dtype_dct = wthr_col_dtypes)

# if sum(checkpoint.Pass)/checkpoint.shape[0] == 1:
#     pass
# else:
#     print(checkpoint.loc[~checkpoint.Pass, ])

## Management

### Date_Datetime

In [ ]:
# mgmt = relocate_to_Imputation_Notes(df = mgmt, col = 'Date_Datetime', val_list= ['Before Planting'])

In [ ]:
# mgmt = sanitize_col(
#     df = mgmt, 
#     col = 'Date_Datetime', 
#     simple_renames= {}, 
#     split_renames= {'6/24/21 for all but plots in pass 2; 7/5/21 for pass 2' : [
#                         '6/24/21 for all but plots in pass 2', '7/5/21 for pass 2']})

In [ ]:
# # make corrections too one-off to fix with a funciton. 
# mask = ((mgmt.Date_Datetime == '6/24/21 for all but plots in pass 2') & (mgmt.Pass != 2.))
# mgmt.loc[mask, 'Date_Datetime'] = '6/24/21'
# # since we split without specifiying pass we need to remove any rows that still have the search string.
# # and overwrite the df
# mask = (mgmt.Date_Datetime == '6/24/21 for all but plots in pass 2')
# mgmt = mgmt.loc[~mask, :].copy()

# mask = ((mgmt.Date_Datetime == '7/5/21 for pass 2') & (mgmt.Pass == 2.))
# mgmt.loc[mask, 'Date_Datetime'] = '7/5/21'
# mask = (mgmt.Date_Datetime == '7/5/21 for pass 2')
# mgmt = mgmt.loc[~mask, :].copy()

In [ ]:
# # convert types
# err_list = find_unconvertable_datetimes(df_col=mgmt.Date_Datetime, pattern='%m/%d/%y', index=False)
# if err_list != []:
#     print(err_list)
# else:
#     mgmt.Date_Datetime = pd.to_datetime(pd.Series(mgmt.Date_Datetime), format = '%m/%d/%y', errors='coerce')

### Amount_Per_Acre

In [ ]:
# mgmt.loc[find_unconvertable_numerics(df_col = mgmt['Amount_Per_Acre'], index = True), ]

In [ ]:
# mgmt = sanitize_col(
#     df = mgmt, 
#     col = 'Amount_Per_Acre', 
#     simple_renames= {'170 lb (actual N)': '170 (N)'}, 
#     split_renames= {'51.75, 40.7, 111.7 (N,P,K)': ['51.75 (N)', '40.7 (P)', '111.7 (K)'],
#                     '31-150-138': ['31 (N)', '150 (P)', '138 (K)'],
#                     '16 (N), 41 (P)': ['16 (N)', '41 (P)']})

In [ ]:
# mgmt = safe_create_col(mgmt, "Ingredient")
# mask = mgmt.Ingredient.isna()
# mgmt.loc[mask, 'Ingredient'] = mgmt.loc[mask, 'Product']

# # assume each string is formated as 'val (key)'. `sanitize_col` should be used to enforce this.
# for e in ['150 (P)', '36.6 (N)', '138 (K)', '111.7 (K)', '41 (P)', '16 (N)', '170 (N)', '35.7 (N)', '51.75 (N)', '31 (N)', '40.7 (P)']:
#     val = re.findall('^\d+[.]*\d*', e)[0]
#     key = re.findall('\(.+\)',      e)[0].replace('(', '').replace(')', '')
    
#     mask = (mgmt['Amount_Per_Acre'] == e)
#     mgmt.loc[mask, 'Ingredient'] = key
#     mgmt.loc[mask, 'Amount_Per_Acre'] = val

In [ ]:
# # convert types
# err_list = find_unconvertable_numerics(df_col = mgmt['Amount_Per_Acre'], index = False)
# if err_list != []:
#     print(err_list)
# else:
#     mgmt.Amount_Per_Acre = pd.to_numeric(mgmt.Amount_Per_Acre, errors='coerce')

### Ingredient
This is to be the cleaned up version of the "Product" column

In [ ]:
# list(mgmt.loc[:, 'Ingredient'].drop_duplicates())

### Simple Columns

In [ ]:
# # to bool
# mgmt['mgmt'] = mgmt['mgmt'].astype('bool')

# # to string
# for e in [ee for ee in ['Application', 'Product', 'Ingredient', 'Unit', 'Imputation_Notes'] if ee in mgmt.columns]:
#     mgmt[e] = mgmt[e].astype('string')
    

# mgmt.Year = year_string
# mgmt.Year = mgmt.Year.astype('string')

### Check Success

In [ ]:
# check_df_dtype_expectations(df = mgmt, dtype_dct = mgmt_col_dtypes)

# Publish



In [ ]:
# write_out_pkl(obj = sval, path = './data/interim/'+year_string+'sval.pickle')
# write_out_pkl(obj = wthr, path = './data/interim/'+year_string+'wthr.pickle')
# write_out_pkl(obj = mgmt, path = './data/interim/'+year_string+'mgmt.pickle')